# Dataset particionado

Se procesa la base de datos de forma tal que todos los samples mayores a 5 segundos, se dividen en segmentos de 5 segundos como máximo.

Dando un total de 56086 instancias

# Predecir un parámetro en función otras variables

Se intentará predecir la Tonalidad del sonido en función de otras variables. Definiendo variables dicotómicas para aquellas variables categóricas que así lo requieran.

In [1]:
import sys
sys.version

'3.7.1 (default, Nov  6 2018, 18:46:03) \n[Clang 10.0.0 (clang-1000.11.45.5)]'

In [2]:
import pandas as pd
import numpy as np

from sklearn import linear_model
import statsmodels.api as sm

np.set_printoptions(precision=2)

In [3]:
data = pd.read_csv('snd-segmented-dataset-from-plain-json.csv')
data.head()

,Duration,Loudness,LogAttackTime,Tempo,Tempo.confidence,TemporalCentroid,SingleEvent,Loop,Tonality,Tonality.confidence,DynamicRange,Note.midi,Note.frequency,Note.confidence,Genre,Mood
0,5.0,-15.304321,0.662552,103,0.0,0.572003,False,False,G major,0.887811,2.226904,66,384.759399,0.000000,Genre A,Mood A
1,5.0,-19.805204,-0.860892,99,0.0,0.539102,False,False,A major,0.798364,1.200455,36,67.521309,0.000000,Genre A,Mood A
2,5.0,-24.740988,-1.379621,116,0.0,0.431652,True,False,F# major,0.494744,2.358749,68,427.887085,0.287887,Genre A,Mood A
3,5.0,-10.838598,0.478131,146,0.0,0.497631,False,False,C major,0.714704,0.501460,60,263.397491,0.000000,Genre A,Mood A
4,5.0,-35.420860,0.560322,98,0.0,0.498890,True,False,A# minor,0.470257,0.576118,47,126.194229,0.623586,Genre B,Mood B


In [4]:
data.shape[0]

56085

In [5]:
# Se descartan columnas que tienen que ver con la confianza
# en las estimaciones en el cálculo de features
# Tip: axis number (0 for rows and 1 for columns)
data = data.drop("Tempo.confidence", axis=1);
data = data.drop("Tonality.confidence", axis=1);
data = data.drop("Note.confidence", axis=1);

### Correlación entre variables

In [6]:
data.corr(method='pearson', min_periods=1) # pearson -> método estándar

,Duration,Loudness,LogAttackTime,Tempo,TemporalCentroid,SingleEvent,Loop,DynamicRange,Note.midi,Note.frequency
Duration,1.000000,0.007492,0.058746,0.019427,0.038723,-0.012383,-0.016708,0.020536,-0.015988,-0.030410
Loudness,0.007492,1.000000,-0.081411,0.046137,0.057581,-0.529873,-0.011885,-0.282938,-0.024508,-0.077803
LogAttackTime,0.058746,-0.081411,1.000000,-0.002584,0.290832,0.070609,-0.037714,-0.049974,-0.020198,-0.024983
Tempo,0.019427,0.046137,-0.002584,1.000000,-0.018764,0.006044,-0.129334,0.042631,-0.002867,-0.006618
TemporalCentroid,0.038723,0.057581,0.290832,-0.018764,1.000000,-0.031623,0.004107,-0.136517,0.006787,0.006423
SingleEvent,-0.012383,-0.529873,0.070609,0.006044,-0.031623,1.000000,-0.015951,0.130924,0.013024,0.034740
Loop,-0.016708,-0.011885,-0.037714,-0.129334,0.004107,-0.015951,1.000000,0.002306,0.005885,0.003960
DynamicRange,0.020536,-0.282938,-0.049974,0.042631,-0.136517,0.130924,0.002306,1.000000,0.093629,0.067941
Note.midi,-0.015988,-0.024508,-0.020198,-0.002867,0.006787,0.013024,0.005885,0.093629,1.000000,0.746004
Note.frequency,-0.030410,-0.077803,-0.024983,-0.006618,0.006423,0.034740,0.003960,0.067941,0.746004,1.000000


**Observación: La correlación entre variables en general es baja, salvo en duración con LogAttackTime, DynamicRange y TemporalCentroid en menor medida.**

## Regresión múltiple (varias variables)

In [7]:
# Referencia: https://stackoverflow.com/questions/11479064/multiple-linear-regression-in-python
def reg_multiple(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results

In [8]:
Duration = np.asarray( data.loc[:, 'Duration' ] )
DynamicRange = np.asarray( data.loc[:, 'DynamicRange' ] )
TemporalCentroid = np.asarray( data.loc[:, 'TemporalCentroid' ] )
LogAttackTime = np.asarray( data.loc[:, 'LogAttackTime' ] )
#Tempo = np.asarray( data.loc[:, 'Tempo' ] )


In [9]:
y = LogAttackTime
X = np.array( [Duration, DynamicRange])

reg_multiple(y, X).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     171.3
Date:                Tue, 19 Feb 2019   Prob (F-statistic):           6.81e-75
Time:                        22:37:12   Log-Likelihood:                -52186.
No. Observations:               56085   AIC:                         1.044e+05
Df Residuals:                   56082   BIC:                         1.044e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0148      0.001    -12.160      0.000      -0.017      -0.012
x2             0.2982      0.021     14.201      0.000       0.257       0.339
const         -1.6277      0.105    -15.518      0.000      -1.833      -1.422
==============================================================================
Omnibus:                     4032.332   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4582.216
Skew:                          -0.675   Prob(JB):                         0.00
Kurtosis:                       2.629   Cond. No.                         220.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Observación: El R cuadrado es muy bajo, cuando se espera algo por lo menos de 0.5.**

## Se agrega el feature TemporalCentroid

In [10]:
y = LogAttackTime
X = np.array( [Duration, DynamicRange, TemporalCentroid])

reg_multiple(y, X).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     1781.
Date:                Tue, 19 Feb 2019   Prob (F-statistic):               0.00
Time:                        22:37:29   Log-Likelihood:                -49806.
No. Observations:               56085   AIC:                         9.962e+04
Df Residuals:                   56081   BIC:                         9.965e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             2.8938      0.041     70.494      0.000       2.813       2.974
x2            -0.0034      0.001     -2.878      0.004      -0.006      -0.001
x3             0.2387      0.020     11.848      0.000       0.199       0.278
const         -2.7929      0.102    -27.413      0.000      -2.993      -2.593
==============================================================================
Omnibus:                     4949.133   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6387.895
Skew:                          -0.824   Prob(JB):                         0.00
Kurtosis:                       3.122   Cond. No.                         224.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Observación: El valor de R cuadrado mejora, pero no se alcanza un valor de p > 0.005 que defina un 95% de confianza.**

## Variables categóricas  y conversión a dummy dicotómicas


**La asignación de un número a cada categoría no resuelve el problema. La solución es crear tantas variables dicotómicas como número de respuestas. Para k valores posibles, se toman k-1 variables dicotómicas.**

### Key o Tonality

Las categorias posibles para la tonalidad se encuentran en notación americana, donde 'A' es La, 'B' es Sí, etc. Se expresan en el array 'key_to_number_list' y luego se mapean en 23 variables dicotómicas, ya que la cantidad de valores posibles es 24.

### Genre, Loop y Mood

Género, si es loopeable y Mood (humor) se mapean en una única variable dicotómica ya que poseen solo dos valores posibles.

In [11]:
key_to_number_list = ['A minor', 'A major', 'A# minor', 'A# major', 'B minor', 'B major', 'C minor', 'C major', 'C# minor', 'C# major', 'D minor', 'D major', 'D# minor', 'D# major','E minor', 'E major', 'F minor', 'F major', 'F# minor', 'F# major', 'G minor', 'G major','G# minor', 'G# major']

def keyToNumber(x_value):
    return [i for i,x in enumerate(key_to_number_list) if x == x_value][0]

In [12]:
len(key_to_number_list)

24

In [13]:
# Mapeo de las tonalidades 'en texto' a categorias numéricas
# Se necesitan k-1 variables dicotómicas, con k=len(key_to_number_list)

# Genre
data['G1'] = data['Genre'].map(lambda x: 1 if x=='Genre A' else 0)

data['L1'] = data['Loop'].map(lambda x: 1 if x==True else 0)
data['M1'] = data['Mood'].map(lambda x: 1 if x=='Mood A' else 0)

# Tonality / Key
data['D1'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==1 else 0)
data['D2'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==2 else 0)
data['D3'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==3 else 0)
data['D4'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==4 else 0)
data['D5'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==5 else 0)
data['D6'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==6 else 0)
data['D7'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==7 else 0)
data['D8'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==8 else 0)
data['D9'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==9 else 0)
data['D10'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==10 else 0)
data['D11'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==11 else 0)
data['D12'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==12 else 0)
data['D13'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==13 else 0)
data['D14'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==14 else 0)

data['D15'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==15 else 0)
data['D16'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==16 else 0)
data['D17'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==17 else 0)
data['D18'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==18 else 0)
data['D19'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==19 else 0)
data['D20'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==20 else 0)
data['D21'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==21 else 0)
data['D22'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==22 else 0)
data['D23'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==23 else 0)

data.head()

,Duration,Loudness,LogAttackTime,Tempo,TemporalCentroid,SingleEvent,Loop,Tonality,DynamicRange,Note.midi,...,D14,D15,D16,D17,D18,D19,D20,D21,D22,D23
0,5.0,-15.304321,0.662552,103,0.572003,False,False,G major,2.226904,66,...,0,0,0,0,0,0,0,1,0,0
1,5.0,-19.805204,-0.860892,99,0.539102,False,False,A major,1.200455,36,...,0,0,0,0,0,0,0,0,0,0
2,5.0,-24.740988,-1.379621,116,0.431652,True,False,F# major,2.358749,68,...,0,0,0,0,0,1,0,0,0,0
3,5.0,-10.838598,0.478131,146,0.497631,False,False,C major,0.501460,60,...,0,0,0,0,0,0,0,0,0,0
4,5.0,-35.420860,0.560322,98,0.498890,True,False,A# minor,0.576118,47,...,0,0,0,0,0,0,0,0,0,0


In [14]:
G1 = np.asarray( data.loc[:, 'G1' ] )
L1 = np.asarray( data.loc[:, 'L1' ] )
M1 = np.asarray( data.loc[:, 'M1' ] )

D1 = np.asarray( data.loc[:, 'D1' ] )
D2 = np.asarray( data.loc[:, 'D2' ] )
D3 = np.asarray( data.loc[:, 'D3' ] )
D4 = np.asarray( data.loc[:, 'D4' ] )
D5 = np.asarray( data.loc[:, 'D5' ] )
D6 = np.asarray( data.loc[:, 'D6' ] )

D7 = np.asarray( data.loc[:, 'D7' ] )
D8 = np.asarray( data.loc[:, 'D8' ] )
D9 = np.asarray( data.loc[:, 'D9' ] )
D10 = np.asarray( data.loc[:, 'D10' ] )
D11 = np.asarray( data.loc[:, 'D11' ] )
D12 = np.asarray( data.loc[:, 'D12' ] )
D13 = np.asarray( data.loc[:, 'D13' ] )
D14 = np.asarray( data.loc[:, 'D14' ] )
D15 = np.asarray( data.loc[:, 'D15' ] )
D16 = np.asarray( data.loc[:, 'D16' ] )
D17 = np.asarray( data.loc[:, 'D17' ] )

D18 = np.asarray( data.loc[:, 'D18' ] )
D19 = np.asarray( data.loc[:, 'D19' ] )
D20 = np.asarray( data.loc[:, 'D20' ] )
D21 = np.asarray( data.loc[:, 'D21' ] )
D22 = np.asarray( data.loc[:, 'D22' ] )

D23 = np.asarray( data.loc[:, 'D23' ] )

In [15]:
# regresión mutivariable con dicotómicas

y = LogAttackTime
X = np.array( [Duration, DynamicRange, TemporalCentroid, G1, L1, M1, D1, D2, D3, D4, D5, D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20,D21,D22,D23])

reg_multiple(y, X).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     199.1
Date:                Tue, 19 Feb 2019   Prob (F-statistic):               0.00
Time:                        22:37:37   Log-Likelihood:                -49699.
No. Observations:               56085   AIC:                         9.946e+04
Df Residuals:                   56056   BIC:                         9.972e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0801      0.019     -4.256      0.000      -0.117      -0.043
x2            -0.0175      0.020     -0.892      0.373      -0.056       0.021
x3            -0.0018      0.011     -0.161      0.872      -0.024       0.020
x4            -0.0174      0.014     -1.201      0.230      -0.046       0.011
x5             0.0344      0.022      1.587      0.113      -0.008       0.077
x6            -0.0426      0.017     -2.563      0.010      -0.075      -0.010
x7            -0.0533      0.015     -3.646      0.000      -0.082      -0.025
x8            -0.0381      0.017     -2.219      0.026      -0.072      -0.004
x9            -0.0046      0.014     -0.333      0.739      -0.032       0.022
x10            0.0034      0.012      0.281      0.779      -0.020       0.027
x11           -0.0753      0.018     -4.087      0.000      -0.111      -0.039
x12           -0.0972      0.023     -4.247      0.000      -0.142      -0.052
x13           -0.0117      0.012     -0.943      0.346      -0.036       0.013
x14           -0.0535      0.013     -4.034      0.000      -0.080      -0.028
x15            0.0143      0.022      0.663      0.507      -0.028       0.057
x16           -0.0347      0.019     -1.852      0.064      -0.071       0.002
x17           -0.0155      0.012     -1.298      0.194      -0.039       0.008
x18            0.0077      0.014      0.550      0.583      -0.020       0.035
x19           -0.0373      0.018     -2.083      0.037      -0.072      -0.002
x20            0.0129      0.015      0.881      0.378      -0.016       0.042
x21            0.0325      0.016      1.980      0.048       0.000       0.065
x22           -0.0198      0.018     -1.101      0.271      -0.055       0.015
x23            0.0192      0.014      1.394      0.163      -0.008       0.046
x24            0.0007      0.003      0.285      0.776      -0.004       0.006
x25           -0.0785      0.008     -9.411      0.000      -0.095      -0.062
x26            0.0007      0.003      0.285      0.776      -0.004       0.006
x27            2.8966      0.041     70.641      0.000       2.816       2.977
x28           -0.0035      0.001     -2.903      0.004      -0.006      -0.001
x29            0.2325      0.020     11.547      0.000       0.193       0.272
const         -2.7425      0.102    -26.820      0.000      -2.943      -2.542
==============================================================================
Omnibus:                     4844.057   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6223.762
Skew:                          -0.815   Prob(JB):                         0.00
Kurtosis:                       3.097   Cond. No.                     1.01e+16
==============================================================================

W

**Observación: El valor de R cuadrado (coeficiente de determinación) mejora.**

## Otra estrategia: limpiar un poco más el dataset

In [16]:
data.shape[0]

56085

### Se filtran los 'single events' ya que se asume que no son canciones

Es decir 'eventos únicos', no van a caracterizar bien tonalidad, género, etc.


In [17]:
data = data[~data['SingleEvent'].isin([True])] 
data = data.drop("SingleEvent", axis=1);

data.shape[0]

35503

### Se mantienen solo los sonidos de más de 120 segundos (canciones )

Criterio: Se filtran los sonidos de más de 2 minutos y menos de 5, para restringir el dataset a canciones más convencionales (menos experimentales, muy cortas o muy largas).

In [18]:
data = data[ data['Duration'] > 60*2 ]
data = data[ data['Duration'] < 60*5 ]
data.shape[0]

0

Queda un total de casi 400 instancias

## Correlación

In [19]:
data.corr(method='pearson', min_periods=1) # pearson -> método estándar

,Duration,Loudness,LogAttackTime,Tempo,TemporalCentroid,Loop,DynamicRange,Note.midi,Note.frequency,G1,...,D14,D15,D16,D17,D18,D19,D20,D21,D22,D23
Duration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Loudness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LogAttackTime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tempo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TemporalCentroid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Loop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DynamicRange,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Note.midi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Note.frequency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Observación: La correlación baja.**

## Otra estrategia: Analizar los single events

In [20]:
data = pd.read_csv('snd-dataset-from-plain-json.csv')
data = data[data['SingleEvent'].isin([True])] 

data.shape[0]

146

In [21]:
# eventos cortos
data = data[ data['Duration'] < 5 ]
data.shape[0]

29

In [22]:
data.corr(method='pearson', min_periods=1) # pearson -> método estándar

,Duration,Loudness,LogAttackTime,Tempo,Tempo.confidence,TemporalCentroid,SingleEvent,Loop,Tonality.confidence,DynamicRange,Note.midi,Note.frequency,Note.confidence
Duration,1.000000,0.409527,0.458463,0.413927,0.109870,-0.003603,NaN,0.111727,0.267255,0.314945,-0.140897,-0.048192,0.070128
Loudness,0.409527,1.000000,0.093151,0.011048,0.254183,-0.259784,NaN,0.254445,0.122818,0.344705,-0.113036,-0.263562,0.137129
LogAttackTime,0.458463,0.093151,1.000000,0.299679,-0.105107,0.646900,NaN,-0.106338,0.292855,-0.274957,0.360239,0.351031,0.465610
Tempo,0.413927,0.011048,0.299679,1.000000,-0.049662,0.220953,NaN,-0.049214,0.106185,0.006667,0.195123,0.199756,0.195006
Tempo.confidence,0.109870,0.254183,-0.105107,-0.049662,1.000000,-0.096474,NaN,0.999910,-0.144554,0.173904,-0.070938,-0.152904,-0.010962
TemporalCentroid,-0.003603,-0.259784,0.646900,0.220953,-0.096474,1.000000,NaN,-0.102161,0.340015,-0.656999,0.343358,0.257487,0.368906
SingleEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Loop,0.111727,0.254445,-0.106338,-0.049214,0.999910,-0.102161,NaN,1.000000,-0.145321,0.180476,-0.070443,-0.152925,-0.012989
Tonality.confidence,0.267255,0.122818,0.292855,0.106185,-0.144554,0.340015,NaN,-0.145321,1.000000,-0.285782,-0.502435,-0.548159,0.184913
DynamicRange,0.314945,0.344705,-0.274957,0.006667,0.173904,-0.656999,NaN,0.180476,-0.285782,1.000000,-0.121501,-0.129814,-0.149882


**Observación:** Se observa que la variable LogAttackTime esta bastante correlacionada linealmente con Duration, TemporaCentroid, por lo cuál se espera una regresión aceptable.

In [23]:
Duration = np.asarray( data.loc[:, 'Duration' ] )
DynamicRange = np.asarray( data.loc[:, 'DynamicRange' ] )
TemporalCentroid = np.asarray( data.loc[:, 'TemporalCentroid' ] )
LogAttackTime = np.asarray( data.loc[:, 'LogAttackTime' ] )
#Tempo = np.asarray( data.loc[:, 'Tempo' ] )

y = LogAttackTime
X = np.array( [Duration, TemporalCentroid])

reg_multiple(y, X).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.631
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     22.21
Date:                Tue, 19 Feb 2019   Prob (F-statistic):           2.37e-06
Time:                        22:38:09   Log-Likelihood:                -19.970
No. Observations:                  29   AIC:                             45.94
Df Residuals:                      26   BIC:                             50.04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.6067      0.663      5.443      0.000       2.245       4.969
x2             0.2551      0.066      3.867      0.001       0.119       0.391
const         -2.9599      0.336     -8.805      0.000      -3.651      -2.269
==============================================================================
Omnibus:                        3.600   Durbin-Watson:                   1.777
Prob(Omnibus):                  0.165   Jarque-Bera (JB):                3.174
Skew:                          -0.786   Prob(JB):                        0.205
Kurtosis:                       2.604   Cond. No.                         22.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Se obtiene un valor aceptable de R cuadrado, pero no así para p**

## Filtrar solo los features calculados con confianza alta

In [26]:
data = pd.read_csv('snd-segmented-dataset-from-plain-json.csv')

#data = data[ data['Tempo.confidence'] > 0.5 ]
data = data[ data['Tonality.confidence'] > 0.8 ]
data = data[ data['Note.confidence'] > 0.5 ]

data.shape[0]

4435

In [27]:
data.corr(method='pearson', min_periods=1) # pearson -> método estándar

,Duration,Loudness,LogAttackTime,Tempo,Tempo.confidence,TemporalCentroid,SingleEvent,Loop,Tonality.confidence,DynamicRange,Note.midi,Note.frequency,Note.confidence
Duration,1.000000,-0.014895,0.037772,0.006670,0.012903,0.002025,NaN,0.012904,0.013516,0.023294,-0.005634,-0.000087,-0.005898
Loudness,-0.014895,1.000000,-0.015329,0.047330,0.003617,0.059265,NaN,0.003239,0.088389,-0.132991,0.001641,-0.096334,-0.118861
LogAttackTime,0.037772,-0.015329,1.000000,-0.022224,-0.030195,0.387117,NaN,-0.030580,0.028958,-0.181088,-0.007041,-0.008770,-0.015761
Tempo,0.006670,0.047330,-0.022224,1.000000,-0.173415,-0.052315,NaN,-0.173607,0.047389,0.013947,-0.016520,-0.036539,0.006702
Tempo.confidence,0.012903,0.003617,-0.030195,-0.173415,1.000000,-0.016273,NaN,0.999966,-0.022747,0.031229,-0.016390,-0.013691,0.024776
TemporalCentroid,0.002025,0.059265,0.387117,-0.052315,-0.016273,1.000000,NaN,-0.017127,0.032315,-0.317591,0.002706,0.012913,-0.010407
SingleEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Loop,0.012904,0.003239,-0.030580,-0.173607,0.999966,-0.017127,NaN,1.000000,-0.022860,0.032553,-0.016229,-0.013400,0.024783
Tonality.confidence,0.013516,0.088389,0.028958,0.047389,-0.022747,0.032315,NaN,-0.022860,1.000000,-0.077086,-0.024952,-0.053733,-0.133210
DynamicRange,0.023294,-0.132991,-0.181088,0.013947,0.031229,-0.317591,NaN,0.032553,-0.077086,1.000000,0.003455,0.014998,0.162174


In [28]:
# Mapeo de las tonalidades 'en texto' a categorias numéricas
# Se necesitan k-1 variables dicotómicas, con k=len(key_to_number_list)

# Genre
data['G1'] = data['Genre'].map(lambda x: 1 if x=='Genre A' else 0)

data['L1'] = data['Loop'].map(lambda x: 1 if x==True else 0)
data['M1'] = data['Mood'].map(lambda x: 1 if x=='Mood A' else 0)

# Tonality / Key
data['D1'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==1 else 0)
data['D2'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==2 else 0)
data['D3'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==3 else 0)
data['D4'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==4 else 0)
data['D5'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==5 else 0)
data['D6'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==6 else 0)
data['D7'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==7 else 0)
data['D8'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==8 else 0)
data['D9'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==9 else 0)
data['D10'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==10 else 0)
data['D11'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==11 else 0)
data['D12'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==12 else 0)
data['D13'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==13 else 0)
data['D14'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==14 else 0)

data['D15'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==15 else 0)
data['D16'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==16 else 0)
data['D17'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==17 else 0)
data['D18'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==18 else 0)
data['D19'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==19 else 0)
data['D20'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==20 else 0)
data['D21'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==21 else 0)
data['D22'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==22 else 0)
data['D23'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==23 else 0)

#data.head()

G1 = np.asarray( data.loc[:, 'G1' ] )
L1 = np.asarray( data.loc[:, 'L1' ] )
M1 = np.asarray( data.loc[:, 'M1' ] )

D1 = np.asarray( data.loc[:, 'D1' ] )
D2 = np.asarray( data.loc[:, 'D2' ] )
D3 = np.asarray( data.loc[:, 'D3' ] )
D4 = np.asarray( data.loc[:, 'D4' ] )
D5 = np.asarray( data.loc[:, 'D5' ] )
D6 = np.asarray( data.loc[:, 'D6' ] )

D7 = np.asarray( data.loc[:, 'D7' ] )
D8 = np.asarray( data.loc[:, 'D8' ] )
D9 = np.asarray( data.loc[:, 'D9' ] )
D10 = np.asarray( data.loc[:, 'D10' ] )
D11 = np.asarray( data.loc[:, 'D11' ] )
D12 = np.asarray( data.loc[:, 'D12' ] )
D13 = np.asarray( data.loc[:, 'D13' ] )
D14 = np.asarray( data.loc[:, 'D14' ] )
D15 = np.asarray( data.loc[:, 'D15' ] )
D16 = np.asarray( data.loc[:, 'D16' ] )
D17 = np.asarray( data.loc[:, 'D17' ] )

D18 = np.asarray( data.loc[:, 'D18' ] )
D19 = np.asarray( data.loc[:, 'D19' ] )
D20 = np.asarray( data.loc[:, 'D20' ] )
D21 = np.asarray( data.loc[:, 'D21' ] )
D22 = np.asarray( data.loc[:, 'D22' ] )

D23 = np.asarray( data.loc[:, 'D23' ] )

In [29]:
Duration = np.asarray( data.loc[:, 'Duration' ] )
DynamicRange = np.asarray( data.loc[:, 'DynamicRange' ] )
LogAttackTime = np.asarray( data.loc[:, 'LogAttackTime' ] )
Loudness = np.asarray( data.loc[:, 'Loudness' ] )
Tempo = np.asarray( data.loc[:, 'Tempo' ] )
TemporalCentroid = np.asarray( data.loc[:, 'TemporalCentroid' ] )

y = LogAttackTime
#X = np.array( [Loudness, TemporalCentroid, Duration] )
X = np.array( [DynamicRange, TemporalCentroid, Duration, G1, L1, M1, D1, D2, D3, D4, D5, D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20,D21,D22,D23])

reg_multiple(y, X).summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     30.48
Date:                Tue, 19 Feb 2019   Prob (F-statistic):          1.09e-146
Time:                        22:38:54   Log-Likelihood:                -3696.7
No. Observations:                4435   AIC:                             7451.
Df Residuals:                    4406   BIC:                             7637.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.1871      0.079     -2.362      0.018      -0.342      -0.032
x2             0.1170      0.087      1.341      0.180      -0.054       0.288
x3             0.0063      0.035      0.178      0.859      -0.063       0.075
x4             0.0034      0.057      0.059      0.953      -0.109       0.116
x5            -0.0563      0.078     -0.719      0.472      -0.210       0.097
x6             0.0317      0.062      0.510      0.610      -0.090       0.153
x7            -0.0519      0.050     -1.045      0.296      -0.149       0.045
x8             0.1118      0.083      1.355      0.176      -0.050       0.274
x9             0.0013      0.045      0.029      0.977      -0.086       0.089
x10            0.0224      0.041      0.549      0.583      -0.058       0.102
x11           -0.0480      0.074     -0.652      0.515      -0.192       0.096
x12            0.0053      0.119      0.045      0.964      -0.229       0.240
x13            0.0097      0.036      0.270      0.787      -0.061       0.080
x14           -0.0120      0.045     -0.268      0.788      -0.100       0.076
x15            0.1172      0.078      1.493      0.135      -0.037       0.271
x16           -0.1688      0.079     -2.137      0.033      -0.324      -0.014
x17           -0.0153      0.039     -0.397      0.691      -0.091       0.060
x18           -0.0770      0.051     -1.509      0.131      -0.177       0.023
x19           -0.0228      0.079     -0.289      0.773      -0.178       0.132
x20            0.0862      0.042      2.045      0.041       0.004       0.169
x21            0.0381      0.053      0.712      0.477      -0.067       0.143
x22            0.0622      0.085      0.728      0.467      -0.105       0.230
x23            0.0219      0.043      0.507      0.612      -0.063       0.106
x24            0.0097      0.009      1.079      0.281      -0.008       0.027
x25           -0.0435      0.028     -1.538      0.124      -0.099       0.012
x26            0.0097      0.009      1.079      0.281      -0.008       0.027
x27            0.2144      0.075      2.868      0.004       0.068       0.361
x28            3.1689      0.126     25.077      0.000       2.921       3.417
x29           -0.0156      0.004     -4.382      0.000      -0.023      -0.009
const         -2.7031      0.380     -7.117      0.000      -3.448      -1.958
==============================================================================
Omnibus:                      485.922   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              658.803
Skew:                          -0.936   Prob(JB):                    8.77e-144
Kurtosis:                       3.250   Cond. No.                     2.25e+15
==============================================================================

W

**Observación: A diferencia de lo que ocurrió con el dataset original, en este caso al filtrar el dataset solo utilizando las instancias que tenian los features calculados solo los features calculados con confianza alta se obtuvieron peores resultados. R cuadrado (ajustado) de 0.157.**


---

**Siguiente:** [7 - DatasetParticionado PCA y t-SNE](7%20-%20DatasetParticionado%20PCA%20y%20t-sne.ipynb)